In [5]:
import pandas as pd
import numpy as np
import itertools
from itertools import combinations, chain
import seaborn as sns

### 1. Load dataset (from csv)

In [6]:
colnames = ['party','d1','d2','size']

k19 = pd.read_csv('input/K19_csv-Table 1.csv',names=colnames)
k20 = pd.read_csv('input/K20_csv-Table 1.csv',names=colnames)

In [36]:
k19.to_csv('input19.csv')

### 2. Get the list of all possible governing coalition proposals (must include formateur, total size > 60)

In [61]:
formateur_id = 0
knesset = k20.copy()

In [62]:
possible_ids = list(range(len(knesset)))

In [63]:
def findsubsets(s, n): 
    return list(map(set, itertools.combinations(s, n))) 

In [64]:
all_combinations = []

for i in range(1,13):
    all_combinations += findsubsets(possible_ids,i)

In [65]:
## validate if formateur is in the proposal, and if total size > 60

valid_proposals = []

for s in all_combinations:
    if sum({knesset.loc[x]['size'] for x in s}) > 60 and formateur_id in s:
        valid_proposals.append(s)

In [66]:
len(all_combinations),len(valid_proposals)

(1023, 387)

### 3. Some preliminary helper functions

In [67]:
# return the group's negotiated position given a set of member indices

def get_negotiated_position(proposal_set):
    aggregated_d1 = {knesset.loc[i]['d1']*knesset.loc[i]['size'] for i in proposal_set}
    aggregated_d2 = {knesset.loc[i]['d2']*knesset.loc[i]['size'] for i in proposal_set}
    
    size = sum({knesset.loc[i]['size'] for i in proposal_set})
    negotiated_d1 = sum(aggregated_d1)/size
    negotiated_d2 = sum(aggregated_d2)/size
    
    return {'d1':negotiated_d1, 'd2':negotiated_d2}

# given a party's agent index, returns its position in dictionary format

def get_position(p):
    d1 = knesset.loc[p]['d1']
    d2 = knesset.loc[p]['d2']
    return {'d1':d1,'d2':d2}

# compute the realizable utility given the negotiated position and the agent's position

def get_ru(n, p):
    d_squared = np.square(n['d1']-p['d1']) + np.square(n['d2']-p['d2'])
    alpha = 2.7
    dimension = 2
    
    return (dimension - alpha*d_squared)

In [68]:
a = get_negotiated_position({0,3,5,6,9})
b = get_position(9)
get_ru(a,b)

-0.2159130018812152

In [69]:
# given a proposal set and value of v_gc, return formateur's expected total payoff and individual payoff

def compute_utilities(proposal_set, vgc):
    proposal_list = list(proposal_set)
    
    np = get_negotiated_position(proposal_set)
    p_list = [{'d1':knesset.loc[p]['d1'], 'd2':knesset.loc[p]['d2']} for p in proposal_list]
    ru_list= [get_ru(np, p) for p in p_list]
    
    size = sum({knesset.loc[i]['size'] for i in proposal_set})
    expected_utility = [r for r in ru_list]
    vgc_payoffs = [vgc*knesset.loc[p]['size']/size for p in proposal_list]
    
    combined_payoff = {}
    expected_payoff = {}
    vgc_payoff = {}
    
    
    for i in range(len(proposal_list)):
        combined_payoff[str(proposal_list[i])] = expected_utility[i] + vgc_payoffs[i]
        #expected_payoff[str(proposal_list[i])] = expected_utility[i]
        #vgc_payoff[str(proposal_list[i])] = vgc_payoffs[i]
    
    formaetur_payoff = combined_payoff[str(formateur_id)]
    aggregated_payoff = sum(combined_payoff.values())
    return {'formaetur_combined_payoff':formaetur_payoff, 'aggregated_payoff':aggregated_payoff, 
            'combined_payoff':combined_payoff,'vgc':vgc}

def all_positive(combined_payoff):
    return all([x>0 for x in list(combined_payoff.values())])

### 4. Computing for utilities

In [70]:
results = []

for p in valid_proposals:
    temp = {}
    temp['proposal'] = p
    for v in [0,1,2,3,5,10]:
        v_key = 'vgc_'+ str(v)
        temp[v_key] = compute_utilities(p,v)
    
    results.append(temp)

In [71]:
f_0 = sorted(results, key = lambda i: i['vgc_0']['formaetur_combined_payoff'],reverse=True)
f_1 = sorted(results, key = lambda i: i['vgc_1']['formaetur_combined_payoff'],reverse=True) 
f_2 = sorted(results, key = lambda i: i['vgc_2']['formaetur_combined_payoff'],reverse=True)
f_3 = sorted(results, key = lambda i: i['vgc_3']['formaetur_combined_payoff'],reverse=True)
f_5 = sorted(results, key = lambda i: i['vgc_5']['formaetur_combined_payoff'],reverse=True)
f_10 = sorted(results, key = lambda i: i['vgc_10']['formaetur_combined_payoff'],reverse=True)


fcp = {'f_0':f_0,'f_1':f_1,'f_2':f_2,'f_3':f_3,'f_5':f_5,'f_10':f_10}

In [72]:
formateur = pd.DataFrame()
formateur['ranking'] = pd.Series(range(1,31))

for f in fcp.keys():
    colname1 = f + '_rank_descending'
    formateur[colname1] = pd.Series([x['proposal'] for x in fcp[f][:30]])
    colname2 = f + '_values'
    vgc_key = 'vgc_' + (f)[2:]
    formateur[colname2] = pd.Series([x[vgc_key]['formaetur_combined_payoff'] for x in fcp[f][:30]])
    colname3 = f + '_una'
    vgc_key = 'vgc_' + (f)[2:]
    formateur[colname3] = pd.Series([all_positive(x[vgc_key]['combined_payoff']) for x in fcp[f][:30]])

In [73]:
formateur

,ranking,f_0_rank_descending,f_0_values,f_0_una,f_1_rank_descending,f_1_values,f_1_una,f_2_rank_descending,f_2_values,f_2_una,f_3_rank_descending,f_3_values,f_3_una,f_5_rank_descending,f_5_values,f_5_una,f_10_rank_descending,f_10_values,f_10_una
0,1,"{0, 3, 4, 5, 7, 8}",1.999613,True,"{0, 3, 5, 6, 7}",2.481295,True,"{0, 3, 5, 6, 7}",2.965166,True,"{0, 4, 5, 6, 7}",3.456952,True,"{0, 4, 5, 6, 7}",4.440559,True,"{0, 4, 5, 6, 7}",6.899575,True
1,2,"{0, 3, 5, 6, 7}",1.997424,True,"{0, 4, 5, 6, 7}",2.473345,True,"{0, 4, 5, 6, 7}",2.965149,True,"{0, 3, 5, 6, 9}",3.452866,True,"{0, 3, 5, 6, 9}",4.436473,True,"{0, 3, 5, 6, 9}",6.895489,True
2,3,"{0, 3, 4, 5, 8}",1.996650,True,"{0, 3, 5, 6, 9}",2.469260,False,"{0, 3, 5, 6, 9}",2.961063,False,"{0, 3, 5, 6, 7}",3.449037,True,"{0, 3, 5, 6, 7}",4.416779,True,"{0, 4, 5, 6, 7, 8}",6.845281,True
3,4,"{0, 3, 4, 5, 6, 7}",1.995283,True,"{0, 3, 5, 6, 7, 8}",2.467698,True,"{0, 3, 5, 6, 7, 8}",2.951569,True,"{0, 3, 5, 6, 7, 8}",3.435440,True,"{0, 3, 5, 6, 7, 8}",4.403182,True,"{0, 3, 5, 6, 7}",6.836134,True
4,5,"{0, 3, 4, 5, 6}",1.993931,True,"{0, 3, 4, 5, 7, 8}",2.461151,True,"{0, 3, 5, 6, 8}",2.936612,True,"{0, 3, 5, 6, 8}",3.420483,True,"{0, 3, 5, 6, 8}",4.388225,True,"{0, 3, 5, 6, 7, 8}",6.822536,True
5,6,"{0, 4, 5, 6, 7, 9}",1.989364,False,"{0, 3, 4, 5, 8}",2.458188,True,"{0, 1, 5}",2.927850,True,"{0, 1, 5}",3.411721,True,"{0, 4, 5, 6, 7, 8}",4.386265,True,"{0, 4, 5, 6, 8}",6.821698,True
6,7,"{0, 3, 4, 5, 6, 7, 8}",1.986163,True,"{0, 3, 5, 6, 8}",2.452741,True,"{0, 3, 4, 5, 7, 8}",2.922690,True,"{0, 4, 5, 6, 7, 8}",3.402658,True,"{0, 1, 5}",4.379463,True,"{0, 1, 6}",6.810347,True
7,8,"{0, 3, 5, 6, 7, 8, 9}",1.983998,False,"{0, 3, 4, 5, 6}",2.448476,True,"{0, 3, 4, 5, 8}",2.919727,True,"{0, 3, 4, 7, 8, 9}",3.390069,True,"{0, 4, 5, 6, 8}",4.362682,True,"{0, 3, 5, 6, 8}",6.807580,True
8,9,"{0, 3, 5, 6, 7, 8}",1.983827,True,"{0, 3, 4, 6, 7, 8}",2.445352,True,"{0, 3, 4, 6, 7, 8}",2.914102,True,"{0, 3, 4, 8, 9}",3.385476,True,"{0, 3, 4, 7, 8, 9}",4.357811,True,"{0, 1, 5}",6.798818,True
9,10,"{0, 3, 4, 5, 6, 7, 8, 9}",1.982414,False,"{0, 1, 5}",2.443980,True,"{0, 4, 5, 6, 7, 8}",2.910855,True,"{0, 3, 4, 5, 7, 8}",3.384228,True,"{0, 3, 4, 8, 9}",4.353218,True,"{0, 3, 4, 7, 8, 9}",6.777166,True


In [74]:
a_0 = sorted(results, key = lambda i: i['vgc_0']['aggregated_payoff'],reverse=True)
a_1 = sorted(results, key = lambda i: i['vgc_1']['aggregated_payoff'],reverse=True) 
a_2 = sorted(results, key = lambda i: i['vgc_2']['aggregated_payoff'],reverse=True)
a_3 = sorted(results, key = lambda i: i['vgc_3']['aggregated_payoff'],reverse=True)
a_5 = sorted(results, key = lambda i: i['vgc_5']['aggregated_payoff'],reverse=True)
a_10 = sorted(results, key = lambda i: i['vgc_10']['aggregated_payoff'],reverse=True)


ap = {'a_0':a_0,'a_1':a_1,'a_2':a_2,'a_3':a_3,'a_5':a_5,'a_10':a_10}

In [75]:
aggregated = pd.DataFrame()
aggregated['ranking'] = pd.Series(range(1,31))

for a in ap.keys():
    colname1 = a + '_rank_descending'
    aggregated[colname1] = pd.Series([x['proposal'] for x in ap[a][:30]])
    colname2 = a + '_values'
    vgc_key = 'vgc_' + a[2:]
    aggregated[colname2] = pd.Series([x[vgc_key]['aggregated_payoff'] for x in ap[a][:30]])
    colname3 = a + '_una'
    vgc_key = 'vgc_' + a[2:]
    aggregated[colname3] = pd.Series([all_positive(x[vgc_key]['combined_payoff']) for x in ap[a][:30]])

In [90]:
for i in range(len(ap['a_0'])):
    if ap['a_0'][i]['proposal'] == {0,4,5,6, 8}:
        print(i)

94


In [94]:
17/len(ap['a_0'])

0.04392764857881137

In [92]:
len(ap['a_0'])

387

### 5. Computing for Robustness

In [24]:
# preliminary helper function, takes a coalition structure and returns all possible one-step reachable states

def is_dissolved(proposal_set):
    size = sum({knesset.loc[i]['size'] for i in proposal_set})
    return size <=60

def generate_simple_n_reachable(proposal_set,n):
    
    n_reachable = []
    
    for i in range(len(proposal_set)-n,len(proposal_set)):
        n_reachable += findsubsets(proposal_set,i)
    
    return n_reachable

def simple_n_robust(proposal_set,n):
    n_reachable = generate_simple_n_reachable(proposal_set,n)
    return 1 - sum([is_dissolved(x) for x in n_reachable]) / len(n_reachable)

In [79]:
simple_n_robust({0,1,3,6},2)

0.19999999999999996

In [80]:
formateur_robustness = formateur.copy()

v_list =  [0,1,2,3,5,10]
colnames1 = ['f_' + str(v) + '_values' for v in v_list]
colnames2 = ['f_' + str(v) + '_una' for v in v_list]

colnames = colnames1 + colnames2
formateur_robustness = formateur_robustness.drop(colnames, 1)

In [81]:
for v in [0,1,2,3,5,10]:
    colname = 'f_' + str(v) + '_rank_descending'
    proposals = list(formateur[colname])
    simple_1_robustness = [simple_n_robust(x,1) for x in proposals]
    simple_2_robustness = [simple_n_robust(x,2) for x in proposals]
    formateur_robustness['f_' + str(v) + '_s1_robust'] = pd.Series(simple_1_robustness)
    formateur_robustness['f_' + str(v) + '_s2_robust'] = pd.Series(simple_2_robustness)

In [82]:
formateur_robustness

,ranking,f_0_rank_descending,f_1_rank_descending,f_2_rank_descending,f_3_rank_descending,f_5_rank_descending,f_10_rank_descending,f_0_s1_robust,f_0_s2_robust,f_1_s1_robust,f_1_s2_robust,f_2_s1_robust,f_2_s2_robust,f_3_s1_robust,f_3_s2_robust,f_5_s1_robust,f_5_s2_robust,f_10_s1_robust,f_10_s2_robust
0,1,"{0, 3, 4, 5, 7, 8}","{0, 3, 5, 6, 7}","{0, 3, 5, 6, 7}","{0, 4, 5, 6, 7}","{0, 4, 5, 6, 7}","{0, 4, 5, 6, 7}",0.333333,0.095238,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2,"{0, 3, 5, 6, 7}","{0, 4, 5, 6, 7}","{0, 4, 5, 6, 7}","{0, 3, 5, 6, 9}","{0, 3, 5, 6, 9}","{0, 3, 5, 6, 9}",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,3,"{0, 3, 4, 5, 8}","{0, 3, 5, 6, 9}","{0, 3, 5, 6, 9}","{0, 3, 5, 6, 7}","{0, 3, 5, 6, 7}","{0, 4, 5, 6, 7, 8}",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.095238
3,4,"{0, 3, 4, 5, 6, 7}","{0, 3, 5, 6, 7, 8}","{0, 3, 5, 6, 7, 8}","{0, 3, 5, 6, 7, 8}","{0, 3, 5, 6, 7, 8}","{0, 3, 5, 6, 7}",0.833333,0.238095,0.333333,0.095238,0.333333,0.095238,0.333333,0.095238,0.333333,0.095238,0.000000,0.000000
4,5,"{0, 3, 4, 5, 6}","{0, 3, 4, 5, 7, 8}","{0, 3, 5, 6, 8}","{0, 3, 5, 6, 8}","{0, 3, 5, 6, 8}","{0, 3, 5, 6, 7, 8}",0.000000,0.000000,0.333333,0.095238,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.095238
5,6,"{0, 4, 5, 6, 7, 9}","{0, 3, 4, 5, 8}","{0, 1, 5}","{0, 1, 5}","{0, 4, 5, 6, 7, 8}","{0, 4, 5, 6, 8}",0.166667,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.095238,0.000000,0.000000
6,7,"{0, 3, 4, 5, 6, 7, 8}","{0, 3, 5, 6, 8}","{0, 3, 4, 5, 7, 8}","{0, 4, 5, 6, 7, 8}","{0, 1, 5}","{0, 1, 6}",0.857143,0.535714,0.000000,0.000000,0.333333,0.095238,0.333333,0.095238,0.000000,0.000000,0.000000,0.000000
7,8,"{0, 3, 5, 6, 7, 8, 9}","{0, 3, 4, 5, 6}","{0, 3, 4, 5, 8}","{0, 3, 4, 7, 8, 9}","{0, 4, 5, 6, 8}","{0, 3, 5, 6, 8}",0.428571,0.214286,0.000000,0.000000,0.000000,0.000000,0.333333,0.095238,0.000000,0.000000,0.000000,0.000000
8,9,"{0, 3, 5, 6, 7, 8}","{0, 3, 4, 6, 7, 8}","{0, 3, 4, 6, 7, 8}","{0, 3, 4, 8, 9}","{0, 3, 4, 7, 8, 9}","{0, 1, 5}",0.333333,0.095238,0.333333,0.095238,0.333333,0.095238,0.000000,0.000000,0.333333,0.095238,0.000000,0.000000
9,10,"{0, 3, 4, 5, 6, 7, 8, 9}","{0, 1, 5}","{0, 4, 5, 6, 7, 8}","{0, 3, 4, 5, 7, 8}","{0, 3, 4, 8, 9}","{0, 3, 4, 7, 8, 9}",0.875000,0.638889,0.000000,0.000000,0.333333,0.095238,0.333333,0.095238,0.000000,0.000000,0.333333,0.095238


In [83]:
# helper function for computing utility-based stabilty concept, takes a utility benchmark, computes loyalty index

def loyal_index(x, benchmark):
    if x <= 0:
        return 0
    else:
        return min(x/benchmark, 1)
    
def look_up_cu(proposal_set, v):
    try:
        return [x['vgc_'+str(v)]['combined_payoff'] for x in results if x['proposal']==proposal_set]
    except:
        return None

def utility_n_robust(proposal_set,n,benchmark,v):
    n_reachable = generate_simple_n_reachable(proposal_set,n)    
    temp = []
        
    utilities_at_v = look_up_cu(proposal_set,v)[0]
    
    for r in n_reachable:
        if not is_dissolved(r):
            temp.append(1)
        else:
            diff = proposal_set - r
            loyal_indices = [loyal_index(utilities_at_v[str(x)],benchmark) for x in diff]
            combined_loyal = np.prod(loyal_indices)
            temp.append(combined_loyal)
                    
    return np.prod(temp)

In [84]:
formateur_robustness_utility = formateur.copy()

v_list =  [0,1,2,3,5,10]
colnames1 = ['f_' + str(v) + '_values' for v in v_list]
colnames2 = ['f_' + str(v) + '_una' for v in v_list]

colnames = colnames1 + colnames2
formateur_robustness_utility = formateur_robustness_utility.drop(colnames, 1)

In [85]:
for v in [0,1,2,3,5,10]:
    
    utility_distribution = [list(x['vgc_'+str(v)]['combined_payoff'].values()) for x in results]
    bench_score = np.quantile(sum(utility_distribution,[]),0.75)
    print(bench_score)
    
    colname = 'f_' + str(v) + '_rank_descending'
    proposals = list(formateur[colname])
    
    utility_1_robustness = [utility_n_robust(x,1,bench_score,v) for x in proposals]
    utility_2_robustness = [utility_n_robust(x,2,bench_score,v) for x in proposals]
    formateur_robustness_utility['f_' + str(v) + '_u1_robust'] = pd.Series(utility_1_robustness)
    formateur_robustness_utility['f_' + str(v) + '_u2_robust'] = pd.Series(utility_2_robustness)

1.824518077903324
2.057694238457317
2.2887064393015737
2.5604614083648434
3.099795387101405
4.402142110412575


In [86]:
formateur_robustness_utility

,ranking,f_0_rank_descending,f_1_rank_descending,f_2_rank_descending,f_3_rank_descending,f_5_rank_descending,f_10_rank_descending,f_0_u1_robust,f_0_u2_robust,f_1_u1_robust,f_1_u2_robust,f_2_u1_robust,f_2_u2_robust,f_3_u1_robust,f_3_u2_robust,f_5_u1_robust,f_5_u2_robust,f_10_u1_robust,f_10_u2_robust
0,1,"{0, 3, 4, 5, 7, 8}","{0, 3, 5, 6, 7}","{0, 3, 5, 6, 7}","{0, 4, 5, 6, 7}","{0, 4, 5, 6, 7}","{0, 4, 5, 6, 7}",0.498252,4.499913e-06,0.128330,3.480483e-05,0.128230,3.466958e-05,0.226570,5.970485e-04,0.181166,1.951575e-04,0.132590,4.097815e-05
1,2,"{0, 3, 5, 6, 7}","{0, 4, 5, 6, 7}","{0, 4, 5, 6, 7}","{0, 3, 5, 6, 9}","{0, 3, 5, 6, 9}","{0, 3, 5, 6, 9}",0.127643,3.388318e-05,0.281456,1.766249e-03,0.262241,1.240233e-03,0.001682,1.348110e-14,0.009029,5.999732e-11,0.020384,3.518984e-09
2,3,"{0, 3, 4, 5, 8}","{0, 3, 5, 6, 9}","{0, 3, 5, 6, 9}","{0, 3, 5, 6, 7}","{0, 3, 5, 6, 7}","{0, 4, 5, 6, 7, 8}",0.128256,3.470524e-05,0.000000,0.000000e+00,0.000000,0.000000e+00,0.119502,2.437130e-05,0.106967,1.400382e-05,0.261923,2.406441e-07
3,4,"{0, 3, 4, 5, 6, 7}","{0, 3, 5, 6, 7, 8}","{0, 3, 5, 6, 7, 8}","{0, 3, 5, 6, 7, 8}","{0, 3, 5, 6, 7, 8}","{0, 3, 5, 6, 7}",1.000000,3.707392e-05,0.188022,1.356185e-07,0.192119,1.251785e-07,0.186008,7.328935e-08,0.177433,3.265871e-08,0.092332,6.710770e-06
4,5,"{0, 3, 4, 5, 6}","{0, 3, 4, 5, 7, 8}","{0, 3, 5, 6, 8}","{0, 3, 5, 6, 8}","{0, 3, 5, 6, 8}","{0, 3, 5, 6, 7, 8}",0.160262,1.057174e-04,0.487100,4.551097e-06,0.087807,5.219774e-06,0.083022,3.944273e-06,0.076485,2.617411e-06,0.169648,1.192523e-08
5,6,"{0, 4, 5, 6, 7, 9}","{0, 3, 4, 5, 8}","{0, 1, 5}","{0, 1, 5}","{0, 4, 5, 6, 7, 8}","{0, 4, 5, 6, 8}",0.239522,0.000000e+00,0.132793,4.129322e-05,0.600417,2.164511e-01,0.587086,2.023511e-01,0.328455,2.267430e-06,0.126263,3.209040e-05
6,7,"{0, 3, 4, 5, 6, 7, 8}","{0, 3, 5, 6, 8}","{0, 3, 4, 5, 7, 8}","{0, 4, 5, 6, 7, 8}","{0, 1, 5}","{0, 1, 6}",1.000000,3.498802e-04,0.086727,4.906610e-06,0.477150,4.345138e-06,0.387770,1.153449e-05,0.568191,1.834354e-01,0.424139,7.630012e-02
7,8,"{0, 3, 5, 6, 7, 8, 9}","{0, 3, 4, 5, 6}","{0, 3, 4, 5, 8}","{0, 3, 4, 7, 8, 9}","{0, 4, 5, 6, 8}","{0, 3, 5, 6, 8}",0.162070,0.000000e+00,0.163157,1.156181e-04,0.136491,4.737154e-05,0.103540,2.502976e-11,0.173096,1.553956e-04,0.070118,1.694926e-06
8,9,"{0, 3, 5, 6, 7, 8}","{0, 3, 4, 6, 7, 8}","{0, 3, 4, 6, 7, 8}","{0, 3, 4, 8, 9}","{0, 3, 4, 7, 8, 9}","{0, 1, 5}",0.183018,1.491683e-07,0.257636,5.704734e-07,0.266517,5.882152e-07,0.018363,2.088129e-09,0.113600,8.813255e-11,0.546651,1.633546e-01
9,10,"{0, 3, 4, 5, 6, 7, 8, 9}","{0, 1, 5}","{0, 4, 5, 6, 7, 8}","{0, 3, 4, 5, 7, 8}","{0, 3, 4, 8, 9}","{0, 3, 4, 7, 8, 9}",1.000000,0.000000e+00,0.605118,2.215743e-01,0.432762,3.368692e-05,0.434419,2.177340e-06,0.023690,7.460976e-09,0.125813,2.971009e-10


In [89]:
formateur_robustness.to_csv('simple20.csv')